In [3]:
import sys
!{sys.executable} -m pip install numpy pandas

  Using cached pandas-1.1.4-cp38-cp38-manylinux1_x86_64.whl (9.3 MB)


In [73]:
import pandas as pd
import numpy as np
pd.low_memory = False

In [68]:
def split_by_nans(df, field):
    non_empty = df[df[field].notnull()]
    nans = df.drop(index=non_empty.index)

    non_empty.reset_index(inplace=True)
    nans.reset_index(inplace=True)
    return non_empty, nans

In [69]:
def split_by_field(df, field, drop=True):
    if drop:
        return {value: sub_df for value, sub_df in df.groupby(field)}
    else:
        non_empty, nans = split_by_nans(df, field)
        ret_val = {'nans': nans}
        ret_val.update({
            value: sub_df for value, sub_df in non_empty.groupby(field)
        })
        return ret_val

In [194]:
def print_counter(data_dict, func=len):
    for k, v in data_dict.items():
        print(f'{k}|{func(v)}')

In [122]:
def year_filter(df, year=2020):
    diagnosis = 'DIAGNOZA-DATA'
    desc_result = 'WYNIK OPISOWY-DATA'
    lab_result = 'WYNIK LABORATORYJNY-DATA'
    start = f'01.01.{year}'
    end = f'01.01.{year + 1}'
    return (
        (df[diagnosis] >= start) & (df[diagnosis] < end) |
        (df[desc_result] >= start) & (df[desc_result] < end) |
        (df[lab_result] >= start) & (df[lab_result] < end)
    )

In [200]:
diagnosis_attributes_map = {
    'Osoba wykonująca badanie': 'OWY',
    'Hemoglobina': 'HGB',
    'Krwinki czerwone': 'RBC',
    'Hematokryt': 'HCT',
    'MCV': 'MCV',
    'MCH': 'MCH',
    'MCHC': 'MCHC',
    'RDW': 'RDW',
    'Płytki krwi': 'PLT',
    'MPV': 'MPV',
    'Wskaźnik anizocytozy płytek krwi': 'PDW',
    'Płytkokryt': 'PLT CT',
    'Odsetek dużych płytek krwi': 'P-LCR',
    'Krwinki białe': 'WBC',
    'Liczba neutrocytów': 'NEUT',
    '% neutrocytów': '% NEUT',
    'Liczba limfocytów': 'LYMPH',
    '% limfocytów': '% LYMPH',
    'Liczba monocytów': 'MONO',
    '% monocytów': '% MONO',
    'Liczba eozynocytów': 'EO',
    '% eozynocytów': '% EO',
    'Liczba bazocytów': 'BAZO',
    '% bazocytów': '% BAZO',
    'IG': 'IG',
    '% niedojrzałych granulocytów': '% IG',
    'NRBC': 'NRBC',
    '% erytoblastów': '%NRBC',
}

In [123]:
data_file = 'data/morphology/morphology.csv'

In [295]:
df = pd.read_csv(data_file, dtype="object")
date_columns = [col for col in df.columns if '-DATA' in col]
print(date_columns)
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# df = df[year_filter(df, year=2020)]

['DIAGNOZA-DATA', 'WYNIK OPISOWY-DATA', 'WYNIK OPISOWY-DATA ZLECENIA', 'WYNIK OPISOWY-DATA OPISU', 'WYNIK OPISOWY-DATA SKIEROWANIA', 'WYNIK OPISOWY-DATA POBRANIA PRÓBKI', 'WYNIK LABORATORYJNY-DATA', 'WYNIK LABORATORYJNY-DATA ZLECENIA', 'WYNIK LABORATORYJNY-DATA OPISU', 'WYNIK LABORATORYJNY-DATA SKIEROWANIA', 'WYNIK LABORATORYJNY-DATA POBRANIA PRÓBKI']


In [296]:
split_dfs = split_by_field(df, 'TYP PARAMETRU')
df_diagnosis = split_dfs['Diagnoza'].dropna(axis=1, how='all')
df_lab_result = split_dfs['Wynik laboratoryjny'].dropna(axis=1, how='all')
df_desc_result = split_dfs['Wynik opisowy'].dropna(axis=1, how='all')

In [297]:
diagnosis_columns = df_diagnosis.columns
lab_result_columns = df_lab_result.columns
desc_result_columns = df_desc_result.columns

In [298]:
def count_empty(df, column):
    counter = df[column].value_counts(dropna=False).to_dict()
    total = len(df)
    empty = 0
    if np.nan in counter:
        empty = counter.pop(np.nan)
    return {
        'non_empty': sum(counter.values()),
        'empty': empty,
        'total': total,
    }


In [299]:
dfs = {
    'diagnosis': (df_diagnosis, diagnosis_columns),
    'lab_result': (df_lab_result, lab_result_columns),
    'desc_result': (df_desc_result, desc_result_columns)
}
for container in dfs.values():
    for column in container[1]:
        counter = count_empty(container[0], column)
        print(f"{column}|{counter['non_empty']}|{counter['empty']}")
    
    print('\n\n\n')

PACJENT-ID|968|0
TYP PARAMETRU|968|0
DIAGNOZA-TYP|968|0
DIAGNOZA-RODZAJ|968|0
DIAGNOZA-KOD ICD|968|0
DIAGNOZA-NAZWA|968|0
DIAGNOZA-DATA|968|0
DIAGNOZA-SZPITAL|968|0
DIAGNOZA-ŹRÓDŁO|968|0




PACJENT-ID|32471|0
TYP PARAMETRU|32471|0
WYNIK LABORATORYJNY-NAZWA BADANIA|32471|0
WYNIK LABORATORYJNY-NAZWA ATRYBUTU|32471|0
WYNIK LABORATORYJNY-KOD ATRYBUTU|32471|0
WYNIK LABORATORYJNY-DATA|32471|0
WYNIK LABORATORYJNY-WARTOŚĆ|32471|0
WYNIK LABORATORYJNY-JEDNOSTKA|25151|7320
WYNIK LABORATORYJNY-NORMA|20040|12431
WYNIK LABORATORYJNY-JEDNOSTKA ZLECAJĄCA|17669|14802
WYNIK LABORATORYJNY-NUMER WYKONANIA|32471|0
WYNIK LABORATORYJNY-DATA ZLECENIA|32471|0
WYNIK LABORATORYJNY-DATA OPISU|32471|0
WYNIK LABORATORYJNY-DATA SKIEROWANIA|32471|0
WYNIK LABORATORYJNY-ZEWNĘTRZNY NUMER BADANIA|32471|0




PACJENT-ID|104|0
TYP PARAMETRU|104|0
WYNIK OPISOWY-NAZWA PROCEDURY|104|0
WYNIK OPISOWY-ATRYBUT FORMULARZA WYNIKOWEGO|104|0
WYNIK OPISOWY-TREŚĆ|104|0
WYNIK OPISOWY-DATA|104|0
WYNIK OPISOWY-OSOBA ZAMAWIAJĄCA|104|0
WYN

In [300]:
lab_result_fields = ['WYNIK LABORATORYJNY-NAZWA BADANIA', 'WYNIK LABORATORYJNY-NAZWA ATRYBUTU', 'WYNIK LABORATORYJNY-KOD ATRYBUTU', 'WYNIK LABORATORYJNY-JEDNOSTKA']

In [301]:
lab_result_attributes = df_lab_result[lab_result_fields]
test_name_dfs = split_by_field(lab_result_attributes, 'WYNIK LABORATORYJNY-NAZWA BADANIA')
test_name_dfs = {k: v for k, v in sorted(test_name_dfs.items(), key=lambda item: - len(item[1]))}
df_morphology = test_name_dfs['Morfologia']

In [302]:
attribute_code_column = 'WYNIK LABORATORYJNY-KOD ATRYBUTU'

attribute_codes_dfs = split_by_field(df_morphology, attribute_code_column)
attribute_codes_lengths = {k: len(v) for k, v in attribute_codes_dfs.items()}

print_counter(attribute_codes_lengths, func=lambda item: item)

% BASO|161
% BAZO|436
% EO|380
% EOS|161
% EOZ|56
% GRAN|217
% IG|315
% LIMF|57
% LYM|160
% LYMPH|380
% MONO|597
% NEUT|380
%IG|137
%NRBC|338
BASO|160
BAZO|437
EO|380
EOS|160
EOZ|57
GRAN|217
HCT|597
HGB|597
IG|452
LIMF|57
LYM|160
LYMPH|380
MCH|597
MCHC|597
MCV|597
MONO|597
MPV|597
NEUT|380
NRBC|338
OWY|331
P-LCR|155
PDW|155
PLT|597
PLT CT|155
RBC|597
RDW|597
WBC|597


In [308]:
attribute_name_column = 'WYNIK LABORATORYJNY-NAZWA ATRYBUTU'

attribute_names_dfs = split_by_field(df_morphology, attribute_name_column)
attribute_names_lengths = {k: len(v) for k, v in attribute_names_dfs.items()}

print_counter(attribute_names_lengths, func= lambda item: item)

% bazocytów|597
% eozynocytów|597
% erytoblastów|338
% limfocytów|597
% monocytów|597
% neutrocytów|597
% niedojrzałych granulocytów|315
%IG|137
Hematokryt|597
Hemoglobina|597
IG|452
Krwinki białe|597
Krwinki czerwone|597
Liczba bazocytów|597
Liczba eozynocytów|597
Liczba limfocytów|597
Liczba monocytów|597
Liczba neutrocytów|597
MCH|597
MCHC|597
MCV|597
MPV|597
NRBC|338
Odsetek dużych płytek krwi|155
Osoba wykonująca badanie|331
Płytki krwi|597
Płytkokryt|155
RDW|597
Wskaźnik anizocytozy płytek krwi|155


In [312]:
print(len(attribute_codes_lengths), len(attribute_names_lengths))

41 29


In [306]:
# sanity check
for name, length in attribute_names_lengths.items():
    if name not in diagnosis_attributes_map:
        raise AttributeError('Key not found, please define it in the map above')
    code = diagnosis_attributes_map[name]
    if length != attribute_codes_lengths[code]:
        raise ValueError(f"Amount of \"{name}\" and it's code (\"{code}\") aren't equal.")    
#     print(name, diagnosis_attributes_map[name], length, attribute_codes_lengths[diagnosis_attributes_map[name]])

ValueError: Amount of "% bazocytów" and it's code ("% BAZO") aren't equal.